In [1]:
from transformers import BertForQuestionAnswering, AutoTokenizer, CamembertForQuestionAnswering, CamembertTokenizer

#English
#modelname = 'deepset/bert-base-cased-squad2'
#model = BertForQuestionAnswering.from_pretrained(modelname)
#tokenizer = AutoTokenizer.from_pretrained(modelname)

#French
modelname = 'etalab-ia/camembert-base-squadFR-fquad-piaf'
model_QA = CamembertForQuestionAnswering.from_pretrained(modelname)
tokenizer = CamembertTokenizer.from_pretrained(modelname)




c:\Users\loren\OneDrive - ENSTA Paris\ENSTA\Q&AML\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#Transform pdf to raw data text
from pdfminer.high_level import extract_text

text = extract_text(r"C:\Users\loren\OneDrive - ENSTA Paris\ENSTA\Q&AML\data\PlaquetteALPHA.pdf")
file = open(r"C:\Users\loren\OneDrive - ENSTA Paris\ENSTA\Q&AML\data\data.txt", "w")
file.write(text)
file.close()

In [2]:
#Document retrieval system ----BM25----
from rank_bm25 import BM25Okapi
import string
import re

import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')

import spacy
#spacy.cli.download("fr_core_news_sm")
nlp_fr = spacy.load("fr_core_news_sm")

dataFile = open(r"C:\Users\loren\OneDrive - ENSTA Paris\ENSTA\Q&AML\data\data_plaquette_ingenieur.txt",'r',encoding='utf-8')
content = dataFile.read()
lines = content.split("\n")
lines = [el for el in lines if el != '']

corpus = lines
stop_words = set(stopwords.words('french'))


def preprocess_text(input_string, string):
    input_string = re.sub("(\w)\'", "",input_string) #ellision
    input_string = input_string.translate(str.maketrans('', '', string.punctuation)).lower() #lowercase and ponctuation removal
    input_string =" ".join([word for word in input_string.split() if word.lower() not in stop_words]) #remove stop words
    tokens = nlp_fr(input_string)
    input_string = " ".join([token.lemma_ for token in tokens])
    return input_string

cleaned_corpus = []

for doc in corpus:
    clean_doc = preprocess_text(doc, string)
    cleaned_corpus.append(clean_doc)


tokenized_corpus = [doc.split(" ") for doc in cleaned_corpus]       

bm25 = BM25Okapi(tokenized_corpus)



In [ ]:
#View results
#print(cleaned_corpus)
print(tokenized_corpus)


In [4]:
#Save model
import pickle
file_name = r"C:\Users\loren\OneDrive - ENSTA Paris\ENSTA\Q&AML\models\sample.pkl"
with open(file_name, 'wb') as file:
    pickle.dump(bm25, file)


In [5]:
#Load model
with open(file_name, 'rb') as file:
    model = pickle.load(file)

In [73]:
#Query Retriever
query = "L'ENSTA propose t il des logements ?"
tokenized_query = preprocess_text(query, string).split(" ")  
retrieved_context = bm25.get_top_n(tokenized_query, corpus, n=10)

In [7]:
from transformers import pipeline
nlp = pipeline('question-answering', model=model_QA, tokenizer=tokenizer)

In [74]:
#CamemBERT answering
#context = " ".join(retrieved_context)
print("Query :", query)
# print("Context :",context)
for context in retrieved_context:
    result = nlp({
        'question': query,
        'context': context
    })
    print(result)


    

Query : L'ENSTA propose t il des logements ?
{'score': 0.14650440216064453, 'start': 0, 'end': 58, 'answer': 'L’École dispose d’une résidence étudiante de 430 logements'}
{'score': 0.19047695398330688, 'start': 160, 'end': 184, 'answer': 'Pour plus d’informations'}
{'score': 0.04595576226711273, 'start': 0, 'end': 11, 'answer': 'Plébiscitée'}
{'score': 0.07958044856786728, 'start': 0, 'end': 37, 'answer': 'De nombreux enseignements de 3e année'}
{'score': 0.018209895119071007, 'start': 377, 'end': 421, 'answer': 'Elle se clôt par la semaine de l’innovation,'}
{'score': 0.2368602305650711, 'start': 0, 'end': 45, 'answer': 'L’ENSTA Paris propose 6 Mastères spécialisés,'}
{'score': 0.09079956263303757, 'start': 28, 'end': 45, 'answer': 'proposé par ENSTA'}
{'score': 0.02829146198928356, 'start': 172, 'end': 191, 'answer': 'Le parcours proposé'}
{'score': 0.39515548944473267, 'start': 325, 'end': 398, 'answer': 'l’association avec le profil « Recherche et Innovation » est recommandée.'}
{'

In [65]:
print(retrieved_context)

['Le diplôme d’ingénieur de l’ENSTA Paris est délivré selon deux voies : la voie étudiante (FISE) et la voie par apprentissage (FISEA). Quelle que soit la voie suivie, c’est bien le même diplôme qui est délivré à l’issue du cursus. L’excellence de la formation reste la même. La 1re année se fait toujours sous statut étudiant', 'Les étudiants issus des CPGE PT et TSI, de licence ou de DUT bénéficient d’un encadrement spécifique et personnalisé : coursTD intégrés pour les enseignements les plus ardus, accompagnement du travail personnel.', 'Les diplômés intègrent des grandes entreprises nationales ou internationales, des PME innovantes, ou se lancent dans l’entrepreneuriat. Environ 80% d’entre eux signent un contrat de travail avant l’obtention du diplôme et 99% travaillent dans les 6 mois après l’obtention du diplôme.']
